# **CNN**


---


**Objective**

The objective of this lab is to implement Convolutional Neural Networks (CNNs) to classify images in the Cats vs. Dogs dataset and the CIFAR-10 dataset. You will explore different configurations by experimenting with:

● 3 Activation Functions

● 3 Weight Initialization Techniques

● 3 Optimizers

Additionally, you will compare your best CNN model for both datasets with a pretrained ResNet-18 model.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import torchvision.models as tv_models
import os, copy, zipfile, urllib.request
from PIL import Image

DATASET_CHOICE = "CATS_DOGS"
BATCH = 32
EPOCHS = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Running on:", DEVICE)

Running on: cpu


In [4]:
def fetch_cats_dogs_dataset(base_dir="./data"):
    dataset_url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"
    zip_file = os.path.join(base_dir, "catsdogs.zip")
    extracted_dir = os.path.join(base_dir, "PetImages")

    os.makedirs(base_dir, exist_ok=True)

    # Download
    if not os.path.exists(zip_file):
        print("Downloading dataset...")
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(dataset_url, zip_file)

    # Extract
    if not os.path.exists(extracted_dir):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_file, 'r') as z:
            z.extractall(base_dir)

    # Remove corrupted images
    print("Cleaning corrupt images...")
    removed = 0
    for label in ["Cat", "Dog"]:
        folder = os.path.join(extracted_dir, label)
        for img_name in os.listdir(folder):
            path = os.path.join(folder, img_name)
            try:
                if os.path.getsize(path) == 0:
                    os.remove(path)
                    removed += 1
                    continue
                with Image.open(path) as im:
                    im.verify()
            except:
                os.remove(path)
                removed += 1
    print(f"Removed {removed} bad files")
    return extracted_dir

In [5]:
def build_dataloaders(name):
    print(f"Loading dataset: {name}")

    if name == "CIFAR10":
        transform = T.Compose([
            T.Resize((32,32)),
            T.ToTensor(),
            T.Normalize((0.5,)*3, (0.5,)*3)
        ])
        train_ds = torchvision.datasets.CIFAR10("./data", train=True, download=True, transform=transform)
        val_ds = torchvision.datasets.CIFAR10("./data", train=False, download=True, transform=transform)
        classes = 10

    else:  # Cats vs Dogs
        path = fetch_cats_dogs_dataset()
        transform = T.Compose([
            T.Resize((64,64)),
            T.ToTensor(),
            T.Normalize((0.5,)*3, (0.5,)*3)
        ])
        full_ds = torchvision.datasets.ImageFolder(path, transform=transform)

        train_len = int(0.8 * len(full_ds))
        val_len = len(full_ds) - train_len
        train_ds, val_ds = torch.utils.data.random_split(full_ds, [train_len, val_len])
        classes = 2

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_ds, batch_size=BATCH)

    return train_loader, val_loader, classes

# **Steps to Complete the Task**

**1.CNN Implementation**

**● Define a CNN architecture:**

o Experiment with different numbers of convolutional layers and filter sizes.

o Include pooling layers and fully connected layers as needed.

o Add dropout and batch normalization for better regularization and stability.

● Experiment with configurations:

o Implement 3 different activation functions:

*   RELU
*   TANH
*   LEAKY RELU



Tanh

Leaky ReLU


o Implement 3 different weight initialization techniques:

* Xavier Initialization

* Kaiming Initialization

* Random Initialization

o Experiment with 3 optimizers:

* SGD

* Adam

* RMSprop

In [6]:
class FlexibleCNN(nn.Module):
    def __init__(self, act_type, init_type, out_classes):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64)
        )
        self.pool = nn.MaxPool2d(2,2)
        self.drop = nn.Dropout(0.5)

        self.flat_size = 64*8*8 if out_classes == 10 else 64*16*16
        self.fc1 = nn.Linear(self.flat_size, 512)
        self.fc2 = nn.Linear(512, out_classes)

        self.activation = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "leaky_relu": nn.LeakyReLU()
        }[act_type]

        self.init_type = init_type
        self._init_weights()

    def forward(self, x):
        x = self.pool(self.activation(self.layer1(x)))
        x = self.pool(self.activation(self.layer2(x)))
        x = x.view(x.size(0), -1)
        x = self.drop(self.activation(self.fc1(x)))
        return self.fc2(x)

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                if self.init_type == "xavier":
                    nn.init.xavier_uniform_(m.weight)
                elif self.init_type == "kaiming":
                    nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                else:
                    nn.init.normal_(m.weight, 0, 0.01)

2.Training and Evaluation

● Train your CNN on each dataset using all combinations of activations, weight initializations, and optimizers.

● Save the best-performing model for each dataset.

● Save the weights of your best-performing models and upload them to a GitHub repository along with your code.

● Use accuracy and loss metrics to evaluate performance.

In [7]:
def train_epoch(model, loader, loss_fn, optimzr):
    model.train()
    total, correct, loss_sum = 0, 0, 0

    for x,y in loader:
        x,y = x.to(DEVICE), y.to(DEVICE)
        optimzr.zero_grad()
        out = model(x)
        loss = loss_fn(out,y)
        loss.backward()
        optimzr.step()

        loss_sum += loss.item()*x.size(0)
        correct += (out.argmax(1)==y).sum().item()
        total += y.size(0)

    return loss_sum/total, correct/total


def validate(model, loader, loss_fn):
    model.eval()
    total, correct, loss_sum = 0, 0, 0

    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = loss_fn(out,y)

            loss_sum += loss.item()*x.size(0)
            correct += (out.argmax(1)==y).sum().item()
            total += y.size(0)

    return loss_sum/total, correct/total

In [8]:
def experiment_custom_cnn(train_loader, val_loader, n_classes):
    acts = ["relu","tanh","leaky_relu"]
    inits = ["xavier","kaiming","random"]
    opts = ["sgd","adam","rmsprop"]

    best_acc, best_cfg, best_state = 0, "", None

    for a in acts:
        for i in inits:
            for o in opts:
                print(f"Config → Act:{a} Init:{i} Opt:{o}")
                net = FlexibleCNN(a,i,n_classes).to(DEVICE)

                optimizer = {
                    "sgd": optim.SGD(net.parameters(), lr=0.01),
                    "adam": optim.Adam(net.parameters(), lr=0.001),
                    "rmsprop": optim.RMSprop(net.parameters(), lr=0.001)
                }[o]

                loss_fn = nn.CrossEntropyLoss()

                for _ in range(EPOCHS):
                    train_epoch(net, train_loader, loss_fn, optimizer)
                    _, val_acc = validate(net, val_loader, loss_fn)

                if val_acc > best_acc:
                    best_acc, best_cfg = val_acc, f"{a}_{i}_{o}"
                    best_state = copy.deepcopy(net.state_dict())
                    print("New Best:", best_acc)

    torch.save(best_state, f"best_cnn_{best_cfg}.pth")
    return best_acc, best_cfg

**3.Transfer Learning with ResNet-18**

● Fine-tune ResNet-18 on both datasets.

● Compare its performance with your best CNN model.

In [9]:
def experiment_resnet(train_loader, val_loader, n_classes):
    print("\nStarting ResNet Transfer Learning")

    model = tv_models.resnet18(weights="DEFAULT")
    for p in model.parameters():
        p.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, n_classes)
    model = model.to(DEVICE)

    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()

    best_acc = 0
    for e in range(EPOCHS):
        train_epoch(model, train_loader, loss_fn, optimizer)
        _, val_acc = validate(model, val_loader, loss_fn)
        print(f"Epoch {e+1} Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_resnet.pth")

    return best_acc

In [11]:
if __name__ == "__main__":
    datasets = ["CATS_DOGS", "CIFAR10"]
    results = {}

    for ds in datasets:
        print(f"\n===== Running on {ds} =====")
        train_loader, val_loader, n_classes = build_dataloaders(ds)

        cnn_acc, cfg = experiment_custom_cnn(train_loader, val_loader, n_classes)
        resnet_acc = experiment_resnet(train_loader, val_loader, n_classes)

        results[ds] = (cnn_acc, resnet_acc, cfg)

    print("\nFINAL RESULTS")
    for ds, (cnn,res,cfg) in results.items():
        print(f"{ds} → CNN({cfg})={cnn:.4f} | ResNet={res:.4f}")


===== Running on CATS_DOGS =====
Loading dataset: CATS_DOGS
Extracting dataset...
Cleaning corrupt images...
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Removed 4 bad files
Config → Act:relu Init:xavier Opt:sgd
New Best: 0.7374
Config → Act:relu Init:xavier Opt:adam
Config → Act:relu Init:xavier Opt:rmsprop
Config → Act:relu Init:kaiming Opt:sgd
Config → Act:relu Init:kaiming Opt:adam
Config → Act:relu Init:kaiming Opt:rmsprop
Config → Act:relu Init:random Opt:sgd
Config → Act:relu Init:random Opt:adam
Config → Act:relu Init:random Opt:rmsprop
Config → Act:tanh Init:xavier Opt:sgd
Config → Act:tanh Init:xavier Opt:adam
Config → Act:tanh Init:xavier Opt:rmsprop
Config → Act:tanh Init:kaiming Opt:sgd
Config → Act:tanh Init:kaiming Opt:adam
Config → Act:tanh Init:kaiming Opt:rmsprop
Config → Act:tanh Init:random Opt:sgd
Config → Act:tanh Init:random Opt:adam
Config → Act:tanh Init:random Opt:rmsprop
Config